In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras 
from keras.models import * 
from keras.layers import * 
from keras.preprocessing import image
import PIL
from keras.callbacks import EarlyStopping

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
!wget http://cb.lk/covid_19
!unzip covid_19

--2022-09-15 12:04:08--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 104.21.72.203, 172.67.187.81, 2606:4700:3031::6815:48cb, ...
Connecting to cb.lk (cb.lk)|104.21.72.203|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2022-09-15 12:04:08--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|104.21.72.203|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2022-09-15 12:04:09--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2022-09-15 12:04

In [ ]:
train_set = train_datagen.flow_from_directory('/content/CovidDataset/Train/',
      target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False)

Found 224 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/CovidDataset/Val/',
     target_size=(224, 224), batch_size=32,class_mode='categorical', shuffle=False)

Found 60 images belonging to 2 classes.


In [ ]:
vgg16_model=keras.applications.vgg16.VGG16()
vgg16_model.summary()
model =keras.Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add (layer)
    layer.trainable=False

model.add(keras.layers.Dense(2, activation='softmax'))


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer="adam", metrics=['accuracy'])

results=model.fit( 
    train_set,  
    epochs=20,
    validation_data=test_set,steps_per_epoch=len(train_set),validation_steps=len(test_set),
    # callbacks=tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
  )


Epoch 1/20
7/7 [==============================] - 15s 1s/step - loss: 2.2921 - accuracy: 0.6429 - val_loss: 1.5078 - val_accuracy: 0.5000
Epoch 2/20
7/7 [==============================] - 7s 1s/step - loss: 2.5930 - accuracy: 0.5000 - val_loss: 0.8328 - val_accuracy: 0.5000
Epoch 3/20
7/7 [==============================] - 7s 1s/step - loss: 0.5218 - accuracy: 0.8170 - val_loss: 1.0086 - val_accuracy: 0.5000
Epoch 4/20
7/7 [==============================] - 7s 949ms/step - loss: 0.9967 - accuracy: 0.4464 - val_loss: 0.7880 - val_accuracy: 0.5000
Epoch 5/20
7/7 [==============================] - 7s 1s/step - loss: 0.6732 - accuracy: 0.6250 - val_loss: 0.4278 - val_accuracy: 0.9000
Epoch 6/20
7/7 [==============================] - 7s 1s/step - loss: 0.4500 - accuracy: 0.8214 - val_loss: 0.5271 - val_accuracy: 0.6333
Epoch 7/20
7/7 [==============================] - 7s 1s/step - loss: 0.4276 - accuracy: 0.8259 - val_loss: 0.4187 - val_accuracy: 0.7833
Epoch 8/20
7/7 [=====================

In [ ]:
model.evaluate(test_set)

2/2 [==============================] - 2s 462ms/step - loss: 0.2009 - accuracy: 0.9667


[0.20087705552577972, 0.9666666388511658]

In [ ]:
model.save('/content/drive/MyDrive/test/vgg16_keras_Covid19.h5')

model1=load_model('/content/drive/MyDrive/test/vgg16_keras_Covid19.h5')

model1.evaluate(test_set)

2/2 [==============================] - 2s 465ms/step - loss: 0.2009 - accuracy: 0.9667


[0.20087705552577972, 0.9666666388511658]